In [19]:
import pandas as pd
import numpy as np

from sklearn import preprocessing, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression

# Generalized Linear Models

### GLM is a good starting point for learning more advanced statisticl modeling. Learning GLM lets you understand how we can use probability distributions as building blocks for modeling. 

## Linear Regression

### Linear regression is used to predict the value of continuous variable y by the linear combination of explanatory variables X.

$\mu_i = b_0 + b_1 x_i$

$y_i ~ \stackrel{}{\sim} N(\mu_i, \epsilon)$

### Here, i indicates the index of each sample. Notice this model assumes normal distribution for the noise term. The model can be illustrated as follows:

![figure](https://miro.medium.com/max/570/0*3OF9LJ_g1Fg_DtT9.png)

## Poisson Regression

### There are several problems if you try to apply linear regression for this kind of data.

1. The relationship between X and Y does not look linear. It’s more likely to be exponential.

2. The variance of Y does not look constant with regard to X. Here, the variance of Y seems to increase when X increases.

3. As Y represents the number of products, it always has to be a positive integer. In other words, Y is a discrete variable. However, the normal distribution used for linear regression assumes continuous variables. This also means the prediction by linear regression can be negative. It’s not appropriate for this kind of count data.

### Here, the more proper model you can think of is the Poisson regression model. Poisson regression is an example of generalized linear models (GLM).

### There are three components in generalized linear models.

1. Linear predictor
2. Link function
3. Probability distribution

![figure2](https://miro.medium.com/max/641/0*-ZviuacMrYAhFbg6.png)

### Linear predictor is just a linear combination of parameter (b) and explanatory variable (x).

### Link function literally “links” the linear predictor and the parameter for probability distribution. In the case of Poisson regression, the typical link function is the log link function. This is because the parameter for Poisson regression must be positive (explained later).

### The last component is the probability distribution which generates the observed variable y. As we use Poisson distribution here, the model is called Poisson regression.

### Now, let's apply Poisson regression to our data. The result should look like this.

![figure](https://miro.medium.com/max/558/0*9XhxRne56Lhvmazv.png)

### The magenta curve is the prediction by Poisson regression. I added the bar plot of the prbability mass function of Poisson distribution to make the difference from linear regression clear.

### The prediction curve is exponential as the inverse of the log link function is an exponential function. From this, it is also clear that the parameter for Poisson regression calculated by the linear predictor guaranteed to be positive.

![figure4](https://miro.medium.com/max/388/0*wNqeqIVx2VRnFjiN.png)

### If you use Python, statsmodels library can be used for GLM. 



In [3]:
# Data process
X = np.array([
    [10, 80], [8, 0], [8, 200], [5, 200], [7, 300], [8, 230], [7, 40], [9, 0], [6, 330], [9, 180]
])
y = np.array([469, 366, 371, 208, 246, 297, 363, 436, 198, 364])

lm = LinearRegression()
lm.fit(X, y)

print(lm.coef_)
print(lm.intercept_ )

[41.51347826 -0.34088269]
65.32391638894836


In [4]:
# Prediction
to_be_predicted = np.array([
    [10, 110]
])

predicted_sales = lm.predict(to_be_predicted)
print(predicted_sales)

[442.96160353]


In [6]:
# Evaluate the performance
mse = np.mean((lm.predict(X) - y) ** 2)
r_squared = lm.score(X, y)
adj_r_squared = r_squared - (1 - r_squared) * (X.shape[1] / (X.shape[0] - X.shape[1] - 1))

print("mse: {}".format(mse))
print("r_squared: {}".format(r_squared))
print("adj_r_squared: {}".format(adj_r_squared))

mse: 417.3006119994701
r_squared: 0.945235852681711
adj_r_squared: 0.929588953447914


In [10]:
# P-value
print(f_regression(X, y)[1])

[0.00051435 0.00844837]


In [21]:
# Build Logistic Regression model
path = "./rsc/train.csv"
titanic_train = pd.read_csv(path)

# Filled empty block(Age) with mdian value
age_median = np.nanmedian(titanic_train["Age"])
new_Age = np.where(titanic_train["Age"].isnull(), age_median, titanic_train["Age"])
titanic_train["Age"] = new_Age

# Create dummy variables
label_encoder = preprocessing.LabelEncoder()
encoded_Sex = label_encoder.fit_transform(titanic_train["Sex"])

# Build train_X
train_X = pd.DataFrame([titanic_train["Pclass"],
                        encoded_Sex,
                        titanic_train["Age"]]).T

# Build Model
logistic_regr = linear_model.LogisticRegression()
logistic_regr.fit(train_X, titanic_train["Survived"])

print(logistic_regr.coef_)
print(logistic_regr.intercept_ )

# Get P-value
print("P-value: {}".format(f_regression(train_X, titanic_train["Survived"])[1]))

# Accuracy
survived_predictions = logistic_regr.predict(train_X)
accuracy = logistic_regr.score(train_X, titanic_train["Survived"])
print("accuracy: {}".format(accuracy))

[[-1.14125534 -2.519584   -0.03271785]]
[4.58517765]
P-value: [2.53704739e-25 1.40606613e-69 5.27606885e-02]
accuracy: 0.7878787878787878
